## Automated Expense Extraction - Receipt Parsing Using YOLO and OCR
### YOLO Model Training 
**Project:** Automated Expense Extraction - Receipt Parsing Using YOLO and OCR


In [1]:
# Install YOLOv8
!pip install ultralytics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from ultralytics import YOLO
from pathlib import Path

In [3]:
# Check if running in Google Colab
if 'COLAB_GPU' in os.environ:
    # Mount Google Drive (for Colab)
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set DATA_PATH for Google Drive
    DATA_PATH = Path('/content/drive/MyDrive/data')
else:
    # Set DATA_PATH for local environment
    DATA_PATH = Path('../data')

# Define YOLO directory
YOLO_DIR = DATA_PATH / "processed/yolo_dataset"

# Verify the YOLO directory
print(f"YOLO Directory: {YOLO_DIR}")
print(f"{'Exists' if YOLO_DIR.exists() else 'Does not exist'}")


YOLO Directory: ../data/processed/yolo_dataset
Exists


In [ ]:
# 1. Load the Model
# For higher accuracy, change to 'yolov8s.pt' (Small) or 'yolov8m.pt' (Medium).
model = YOLO('yolov8n.pt')
# model = YOLO('yolov8s.pt')

# 2. Train the Model with HIGH RESOLUTION
print("Starting High-Res Training...")
results = model.train(
    data=str(YOLO_DIR / "dataset.yaml"),
    epochs=2,                   
    imgsz=1280,                  
    rect=True,                   
    batch=4,                     
    project=str(DATA_PATH / "models"),
    name="yolo_receipts_highres_nano",
    exist_ok=True
)

print("High-Res Training Complete!")

print("Training Complete!")
print(f"Best Model Saved at: {DATA_PATH}/models/yolo_receipts_highres_nano/weights/best.pt")

Starting High-Res Training...
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.231 🚀 Python-3.12.2 torch-2.9.1 CPU (Apple M1 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/processed/yolo_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_recei

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# 1. Locate the Results CSV
# Matches the 'project' and 'name' arguments from your training block
run_dir = DATA_PATH / "models/yolo_receipts_highres_nano"
csv_path = run_dir / "results.csv"

if csv_path.exists():
    # Read Data
    df = pd.read_csv(csv_path)
    # YOLO column names often have spaces, strip them
    df.columns = df.columns.str.strip()

    # 2. Setup Plots
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # --- Plot 1: Training & Val Loss ---
    # Box Loss = How well it draws the box
    # Cls Loss = How well it knows "Vendor" vs "Total"
    axs[0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', linestyle='--')
    axs[0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', linewidth=2)
    axs[0].plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss', linestyle='--')
    axs[0].plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss', linewidth=2)
    axs[0].set_title('Loss Metrics (Lower is Better)')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)

    # --- Plot 2: Mean Average Precision (mAP) ---
    # mAP50 = Accuracy at 50% Intersection over Union
    # mAP50-95 = Robust accuracy (averaged over many overlap thresholds)
    axs[1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50', color='green', linewidth=2)
    axs[1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@50-95', color='blue', linewidth=2)
    axs[1].set_title('Precision/Recall Metrics (Higher is Better)')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy (0-1)')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print(f"Results file not found at {csv_path}. Did training finish?")

In [6]:
from IPython.display import Image, display

# YOLO automatically generates this matrix
matrix_path = run_dir / "confusion_matrix.png"

if matrix_path.exists():
    print("--- Confusion Matrix ---")
    display(Image(filename=matrix_path, width=600))
else:
    print("Confusion matrix not found (requires validation run).")

NameError: name 'run_dir' is not defined

In [ ]:
# Show Ground Truth 
val_labels_path = run_dir / "val_batch0_labels.jpg"
# Show Predictions 
val_pred_path = run_dir / "val_batch0_pred.jpg"

if val_pred_path.exists() and val_labels_path.exists():
    print("--- Ground Truth (Left) vs. Predictions (Right) ---")
    
    # Display side by side using matplotlib
    fig, ax = plt.subplots(1, 2, figsize=(18, 10))
    
    img_lbl = plt.imread(val_labels_path)
    img_pred = plt.imread(val_pred_path)
    
    ax[0].imshow(img_lbl)
    ax[0].set_title("Ground Truth Annotations")
    ax[0].axis('off')
    
    ax[1].imshow(img_pred)
    ax[1].set_title("Model Predictions")
    ax[1].axis('off')
    
    plt.show()